In [29]:
from laserembeddings import Laser
from os import path
import pandas as pd
from tqdm import tqdm_notebook,tqdm
#import demoji
#demoji.download_codes()
%load_ext autoreload
%autoreload 2
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
!

^C
ERROR: Operation cancelled by user


In [4]:
parent_path='../Data/New_Data_15-06-2020/'

In [ ]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')
# annotated_df=remove_duplicates_within(annotated_df)
print(len(annotated_df))

In [ ]:
list_text = []
lang_text = []
for index,row in tqdm(annotated_df.iterrows(),total=len(annotated_df)):
    if len(row['message_text'])>4000:
        string=row['message_text'][0:2000]+row['message_text'][-2000:]
        list_text.append(demoji.replace(string,repl=""))
    else:
        list_text.append(demoji.replace(row["message_text"],repl=""))
    lang_text.append(row["language"])

### LASER

In [ ]:
laser = Laser()

embed_list=[]

length_given=len(list_text)
batch_size=64
for i in tqdm(range(0,length_given,batch_size)):
    if(i+batch_size<=length_given):
        temp_text=list_text[i:i+batch_size]
        temp_lang=lang_text[i:i+batch_size]
    else:
        temp_text=list_text[i:length_given]
        temp_lang=lang_text[i:length_given]
    embeddings = laser.embed_sentences(temp_text,lang=temp_lang)  # lang is only used for tokenization
    embed_list+=list(embeddings)

In [ ]:
X_0 = np.array(embed_list)
y_0 = np.array(annotated_df['one_fear_speech'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import random

In [ ]:
acc=[]
macro_f1=[]
prec=[]
recall=[]
prob=[]

skf = StratifiedKFold(n_splits=10, random_state= 2020)

for train_index, test_index in skf.split(X_0, y_0):
    print("TRAIN:", train_index[0:5], "TEST:", test_index[0:5])
    X_train, X_test = X_0[train_index], X_0[test_index]
    y_train, y_test = y_0[train_index], y_0[test_index]
    class_weights = dict(zip(np.unique(y_train), compute_class_weight("balanced", np.unique(y_train),y_train)))
    print(class_weights)
    classifier= LogisticRegression(class_weight=class_weights)
    #classifier=SVC(class_weight=class_weights,kernel='rbf',probability=True)
    classifier.fit(X_train, y_train)
    y_pred=classifier.predict(X_test)
    acc.append(accuracy_score(y_test, y_pred))
    macro_f1.append(f1_score(y_test, y_pred, average='macro'))
    prec.append(precision_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    prob.append(classifier.predict_proba(X_test))


### For SVC (with rbf kernel)

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))

### For logisitic regression

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))

### Doc2Vec


In [ ]:
from utils.preprocess import *
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

list_sents=[preprocess_sent(ele,params={'remove_numbers': True, 'remove_emoji': True, 'remove_stop_words': False, 'tokenize': True}) for ele in tqdm(annotated_df['message_text'],total=len(annotated_df))]
X_0 = np.array(list_sents,dtype='object')
y_0 = np.array(annotated_df['one_fear_speech'])

In [ ]:
acc=[]
macro_f1=[]
prec=[]
recall=[]
prob=[]

skf = StratifiedKFold(n_splits=10, random_state= 2020)

for train_index, test_index in skf.split(X_0, y_0):
    print("TRAIN:", train_index[0:5], "TEST:", test_index[0:5])
    X_train, X_test = X_0[train_index], X_0[test_index]
    y_train, y_test = y_0[train_index], y_0[test_index]
    class_weights = dict(zip(np.unique(y_train), compute_class_weight("balanced", np.unique(y_train),y_train)))
    
    
    print(class_weights)
    
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(X_train)]
    model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=10)
    X_train_embed = np.array([list(model.infer_vector(ele)) for ele in X_train])
    X_test_embed = np.array([list(model.infer_vector(ele)) for ele in X_test])
    #classifier= LogisticRegression(class_weight=class_weights,max_iter=500)
    classifier=SVC(class_weight=class_weights,kernel='rbf',probability=True)
    classifier.fit(X_train_embed, y_train)
    y_pred=classifier.predict(X_test_embed)
    acc.append(accuracy_score(y_test, y_pred))
    macro_f1.append(f1_score(y_test, y_pred, average='macro'))
    prec.append(precision_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    prob.append(classifier.predict_proba(X_test_embed))


### SVC


In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))

### Logistic Regression

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))

In [ ]:
documents

In [5]:
import pandas as pd
from utils.preprocess import *
 

annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')
params_preprocess={'remove_numbers': True, 'remove_emoji': True, 'remove_stop_words': False, 'tokenize': False}
list_sents=[preprocess_doc(ele,params=params_preprocess) for ele in tqdm(annotated_df['message_text'],total=len(annotated_df))]

100%|██████████| 4782/4782 [00:38<00:00, 124.63it/s]


In [14]:
length_sent=[]
for sent in list_sents:
    
    length_sent.append(len(sent))

In [26]:
np.where(np.array(length_sent) > 80)[0].shape

(102,)

In [16]:
length_sent

[1,
 4,
 1,
 1,
 16,
 10,
 29,
 2,
 11,
 25,
 1,
 2,
 5,
 15,
 1,
 17,
 1,
 1,
 30,
 24,
 11,
 1,
 8,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 12,
 4,
 1,
 5,
 53,
 4,
 32,
 98,
 28,
 29,
 5,
 1,
 1,
 1,
 1,
 18,
 19,
 26,
 4,
 13,
 7,
 2,
 4,
 7,
 44,
 4,
 17,
 1,
 1,
 1,
 1,
 2,
 20,
 33,
 5,
 15,
 3,
 22,
 43,
 1,
 2,
 5,
 26,
 12,
 33,
 1,
 1,
 1,
 7,
 4,
 15,
 16,
 1,
 14,
 8,
 1,
 2,
 5,
 8,
 8,
 1,
 2,
 1,
 1,
 2,
 11,
 1,
 45,
 3,
 10,
 24,
 8,
 2,
 9,
 30,
 6,
 18,
 4,
 7,
 23,
 1,
 17,
 18,
 1,
 1,
 5,
 39,
 20,
 27,
 6,
 2,
 1,
 1,
 56,
 1,
 8,
 1,
 9,
 2,
 24,
 1,
 1,
 1,
 8,
 1,
 1,
 2,
 9,
 15,
 1,
 27,
 1,
 4,
 1,
 9,
 8,
 5,
 15,
 65,
 23,
 1,
 73,
 5,
 2,
 2,
 1,
 2,
 1,
 1,
 11,
 117,
 3,
 1,
 5,
 1,
 13,
 55,
 26,
 1,
 14,
 77,
 2,
 2,
 11,
 23,
 10,
 5,
 1,
 5,
 11,
 18,
 12,
 1,
 1,
 2,
 1,
 14,
 4,
 30,
 1,
 4,
 16,
 117,
 1,
 1,
 1,
 7,
 1,
 14,
 6,
 14,
 12,
 1,
 1,
 4,
 1,
 2,
 51,
 11,
 41,
 13,
 10,
 1,
 9,
 37,
 1,
 1,
 15,
 78,
 5,
 1,
 46,
 38,
 1,
 1,
 1,
 1,